# Revised Dual Simplex Algorithm

This notebook implements the revised dual simplex algorithm for solving linear programming problems. The revised dual simplex method is particularly efficient for problems where we have an initial solution that is dual feasible but primal infeasible.

## Algorithm Overview

The revised dual simplex algorithm works as follows:

1. Start with a dual feasible solution (primal may be infeasible)
2. Iteratively improve the solution by:
   - Identifying a basic variable that violates primal feasibility
   - Finding a non-basic variable to enter the basis
   - Updating the basis and solution
3. Continue until primal feasibility is achieved

## Implementation

We'll implement the algorithm following the pseudocode provided, with detailed explanations of each step.


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List, Optional, Dict, Any
import sys
import os
from pathlib import Path
project_root = str(Path(__file__).parent.parent)
sys.path.append(project_root)
from utils.visualization import plot_2d_lp



NameError: name '__file__' is not defined

## Revised Dual Simplex Algorithm Implementation

We'll implement the algorithm based on the provided pseudocode:


In [ ]:
def revised_dual_simplex(A: np.ndarray, b: np.ndarray, c: np.ndarray, 
                         B_indices: List[int], N_indices: List[int],
                         max_iterations: int = 100, 
                         tolerance: float = 1e-10) -> Tuple[str, np.ndarray, np.ndarray, List[Dict[str, Any]]]:
    """
    Solve a linear program using the revised dual simplex method.
    
    Parameters
    ----------
    A : np.ndarray
        Constraint matrix (m x n)
    b : np.ndarray
        Right-hand side vector (m x 1)
    c : np.ndarray
        Cost vector (n x 1)
    B_indices : List[int]
        List of indices for the basic variables (length m)
    N_indices : List[int]
        List of indices for the non-basic variables (length n - m)
    max_iterations : int
        Maximum number of iterations
    tolerance : float
        Numerical tolerance for zero
        
    Returns
    -------
    Tuple[str, np.ndarray, np.ndarray, List[Dict[str, Any]]]
        Status message, optimal solution x, dual solution y, and iteration history
    """
    # Get dimensions
    m, n = A.shape
    
    # Initialize iteration history
    history = []
    
    # Verify input dimensions
    assert b.shape == (m,) or b.shape == (m, 1), "b must be an m-dimensional vector"
    assert c.shape == (n,) or c.shape == (n, 1), "c must be an n-dimensional vector"
    assert len(B_indices) == m, "B_indices must have length m"
    assert len(N_indices) == n - m, "N_indices must have length n - m"
    assert set(B_indices + N_indices) == set(range(n)), "B_indices and N_indices must partition the set of indices"
    
    # Ensure b and c are column vectors
    if b.ndim == 1:
        b = b.reshape(-1, 1)
    if c.ndim == 1:
        c = c.reshape(-1, 1)
    
    # Initialize iteration counter
    iteration = 0
    
    # Step 1: Initialize
    # Extract the basis matrix B and non-basis matrix N
    B = A[:, B_indices]
    N = A[:, N_indices]
    
    # Compute the basis inverse
    B_inv = np.linalg.inv(B)
    
    # Compute the basic variable values
    x_B = B_inv @ b
    
    # Compute the dual solution
    c_B = c[B_indices]
    y = (B_inv.T @ c_B).reshape(-1, 1)
    
    # Compute the dual slacks
    c_N = c[N_indices]
    s_N = c_N - N.T @ y
    
    # Record initial state
    history.append({
        'iteration': iteration,
        'B_indices': B_indices.copy(),
        'N_indices': N_indices.copy(),
        'x_B': x_B.copy(),
        'y': y.copy(),
        's_N': s_N.copy()
    })
    
    # Main loop
    while iteration < max_iterations:
        iteration += 1
        
        # Step 2: Check Primal Feasibility
        if np.all(x_B >= -tolerance):
            # Optimal solution found
            # Construct the full solution vector
            x_full = np.zeros((n, 1))
            x_full[B_indices] = x_B
            return "Optimal solution found", x_full, y, history
        
        # Find the most negative basic variable
        k_row = np.argmin(x_B)
        k = B_indices[k_row]
        
        # Step 3: Compute the Dual Direction
        # Extract the k-th row of B_inv
        e_k = np.zeros((m, 1))
        e_k[k_row] = 1
        u = B_inv.T @ e_k  # This is the k-th row of B_inv
        
        # Compute the direction vector
        d = N.T @ u
        
        # Step 4: Choose Entering Variable
        # Check if there are any negative entries in d
        if not np.any(d < -tolerance):
            return "Dual unbounded, primal infeasible", None, None, history
        
        # Perform the ratio test
        ratios = np.zeros(len(N_indices))
        for i in range(len(N_indices)):
            if d[i] < -tolerance:
                ratios[i] = s_N[i] / (-d[i])
            else:
                ratios[i] = np.inf
        
        # Find the minimum ratio
        p_col = np.argmin(ratios)
        p = N_indices[p_col]
        
        # Step 5: Update the Basis
        # Swap indices
        B_indices[k_row] = p
        N_indices[p_col] = k
        
        # Step 6: Update Inverse Efficiently
        # Compute the pivot column
        a_p = A[:, p].reshape(-1, 1)
        pivot_col = B_inv @ a_p
        
        # Compute the pivot element
        pivot_element = pivot_col[k_row, 0]
        
        # Update the basis inverse using the Sherman-Morrison-Woodbury formula
        # B_inv_new = B_inv + (B_inv @ a_p - e_k) @ (e_k.T @ B_inv) / pivot_element
        B_inv_row_k = B_inv[k_row, :].reshape(1, -1)
        update_term = (pivot_col - e_k) @ B_inv_row_k / pivot_element
        B_inv = B_inv - update_term
        
        # Step 7: Update Dual and Primal Variables
        # Update the basis matrix B and non-basis matrix N
        B = A[:, B_indices]
        N = A[:, N_indices]
        
        # Update the basic variable values
        x_B = B_inv @ b
        
        # Update the dual solution
        c_B = c[B_indices]
        y = (B_inv.T @ c_B).reshape(-1, 1)
        
        # Update the dual slacks
        c_N = c[N_indices]
        s_N = c_N - N.T @ y
        
        # Record this iteration
        history.append({
            'iteration': iteration,
            'B_indices': B_indices.copy(),
            'N_indices': N_indices.copy(),
            'x_B': x_B.copy(),
            'y': y.copy(),
            's_N': s_N.copy(),
            'leaving_index': k,
            'entering_index': p
        })
    
    # If we reach here, we've hit the maximum number of iterations
    return f"Maximum iterations ({max_iterations}) reached without convergence", None, None, history


## Example 1: A Simple Linear Program

Let's test our implementation on a simple linear program:


In [ ]:
# Define a simple linear program
# min  -2x1 - 3x2
# s.t.  x1 + 2x2 <= 4
#       2x1 + x2 <= 4
#       x1, x2 >= 0

# Convert to standard form by adding slack variables
# min  -2x1 - 3x2 + 0x3 + 0x4
# s.t.  x1 + 2x2 + x3 = 4
#       2x1 + x2 + x4 = 4
#       x1, x2, x3, x4 >= 0

# Define the problem data
A = np.array([
    [1, 2, 1, 0],
    [2, 1, 0, 1]
])

b = np.array([4, 4])

c = np.array([-2, -3, 0, 0])

# Initial basis: slack variables
B_indices = [2, 3]  # Indices of the slack variables
N_indices = [0, 1]  # Indices of the original variables

# Solve using the revised dual simplex method
status, x_opt, y_opt, history = revised_dual_simplex(A, b, c, B_indices, N_indices)

print(f"Status: {status}")
print(f"Optimal solution: x = {x_opt.flatten()}")
print(f"Optimal objective value: {np.dot(c, x_opt)}")
print(f"Dual solution: y = {y_opt.flatten()}")


## Example 2: A Problem Requiring Dual Simplex

Now let's try a problem where the initial solution is dual feasible but primal infeasible, which is the typical use case for the dual simplex method:


In [ ]:
# Define a linear program where dual simplex is appropriate
# min  2x1 + 3x2
# s.t.  x1 + x2 >= 5
#       x1 + 2x2 >= 6
#       x1, x2 >= 0

# Convert to standard form by adding surplus variables and negating constraints
# min  2x1 + 3x2 + 0x3 + 0x4
# s.t. -x1 - x2 + x3 = -5
#      -x1 - 2x2 + x4 = -6
#       x1, x2, x3, x4 >= 0

# Define the problem data
A = np.array([
    [-1, -1, 1, 0],
    [-1, -2, 0, 1]
])

b = np.array([-5, -6])

c = np.array([2, 3, 0, 0])

# Initial basis: surplus variables
B_indices = [2, 3]  # Indices of the surplus variables
N_indices = [0, 1]  # Indices of the original variables

# Solve using the revised dual simplex method
status, x_opt, y_opt, history = revised_dual_simplex(A, b, c, B_indices, N_indices)

print(f"Status: {status}")
if x_opt is not None:
    print(f"Optimal solution: x = {x_opt.flatten()}")
    print(f"Optimal objective value: {np.dot(c, x_opt)}")
    print(f"Dual solution: y = {y_opt.flatten()}")
    
    # Print the iteration history
    print("\nIteration History:")
    for i, iter_data in enumerate(history):
        print(f"Iteration {iter_data['iteration']}:")
        print(f"  Basic indices: {iter_data['B_indices']}")
        print(f"  Basic variables: {iter_data['x_B'].flatten()}")
        if i > 0:
            print(f"  Leaving index: {iter_data['leaving_index']}")
            print(f"  Entering index: {iter_data['entering_index']}")
        print()


## Visualizing the Solution

For 2D problems, we can visualize the solution and the path taken by the dual simplex algorithm:


In [ ]:
def visualize_dual_simplex_path(A, b, c, history, original_variables=[0, 1]):
    """
    Visualize the path taken by the dual simplex algorithm for a 2D problem.
    
    Parameters
    ----------
    A : np.ndarray
        Constraint matrix
    b : np.ndarray
        Right-hand side vector
    c : np.ndarray
        Cost vector
    history : List[Dict[str, Any]]
        Iteration history from the revised_dual_simplex function
    original_variables : List[int]
        Indices of the original variables to plot (default: [0, 1])
    """
    # Extract the constraints for the original variables
    constraints = []
    for i in range(A.shape[0]):
        # Extract coefficients for the original variables
        coeffs = A[i, original_variables]
        rhs = b[i]
        
        # Determine the constraint type based on the sign of the coefficients
        if np.all(coeffs <= 0):
            # If all coefficients are non-positive, it's a >= constraint
            constraint_type = '>='
            # Negate coefficients and RHS to make it a <= constraint for plotting
            coeffs = -coeffs
            rhs = -rhs
        else:
            # Otherwise, assume it's a <= constraint
            constraint_type = '<='
        
        constraints.append({
            'coeffs': tuple(coeffs),
            'rhs': rhs,
            'type': constraint_type,
            'label': f'Constraint {i+1}'
        })
    
    # Define the objective function
    objective = {
        'coeffs': tuple(c[original_variables]),
        'sense': 'min',
        'contour_values': np.linspace(0, 20, 5)
    }
    
    # Extract the path of basic feasible solutions
    path = []
    for i, iter_data in enumerate(history):
        # Check if both original variables are in the basis
        x_full = np.zeros(len(c))
        for j, idx in enumerate(iter_data['B_indices']):
            x_full[idx] = iter_data['x_B'][j, 0]
        
        # Extract the values of the original variables
        point = tuple(x_full[original_variables])
        
        # Calculate the objective value
        obj_value = np.dot(c, x_full)
        
        path.append({
            'point': point,
            'objective_value': obj_value,
            'iteration': iter_data['iteration']
        })
    
    # Determine the optimal point (last iteration)
    if len(path) > 0:
        optimal_point = path[-1]['point']
    else:
        optimal_point = None
    
    # Plot the LP and the path
    from utils.visualization import plot_simplex_iterations
    
    fig, ax = plot_simplex_iterations(
        iterations=path,
        constraints=constraints,
        objective=objective,
        x_range=(0, 10),
        y_range=(0, 10),
        title="Dual Simplex Method Path"
    )
    
    plt.show()
    
    return fig, ax

# Visualize the solution path for Example 2
# First, convert the problem back to the original form for visualization
# min  2x1 + 3x2
# s.t.  x1 + x2 >= 5
#       x1 + 2x2 >= 6
#       x1, x2 >= 0

A_viz = np.array([
    [1, 1],
    [1, 2]
])

b_viz = np.array([5, 6])

c_viz = np.array([2, 3])

# Visualize the path
fig, ax = visualize_dual_simplex_path(A_viz, b_viz, c_viz, history)


## Example 3: A Larger Problem

Let's test our implementation on a larger problem:


In [ ]:
# Define a larger linear program
# min  3x1 + 2x2 + x3
# s.t.  x1 + x2 + x3 >= 10
#       2x1 + x2 >= 8
#       x1 + 2x3 >= 7
#       x1, x2, x3 >= 0

# Convert to standard form by adding surplus variables and negating constraints
# min  3x1 + 2x2 + x3 + 0x4 + 0x5 + 0x6
# s.t. -x1 - x2 - x3 + x4 = -10
#      -2x1 - x2 + x5 = -8
#      -x1 - 2x3 + x6 = -7
#       x1, x2, x3, x4, x5, x6 >= 0

# Define the problem data
A = np.array([
    [-1, -1, -1, 1, 0, 0],
    [-2, -1, 0, 0, 1, 0],
    [-1, 0, -2, 0, 0, 1]
])

b = np.array([-10, -8, -7])

c = np.array([3, 2, 1, 0, 0, 0])

# Initial basis: surplus variables
B_indices = [3, 4, 5]  # Indices of the surplus variables
N_indices = [0, 1, 2]  # Indices of the original variables

# Solve using the revised dual simplex method
status, x_opt, y_opt, history = revised_dual_simplex(A, b, c, B_indices, N_indices)

print(f"Status: {status}")
if x_opt is not None:
    print(f"Optimal solution: x = {x_opt.flatten()}")
    print(f"Optimal objective value: {np.dot(c, x_opt)}")
    print(f"Dual solution: y = {y_opt.flatten()}")


## Conclusion

In this notebook, we've implemented the revised dual simplex algorithm for solving linear programming problems. The algorithm is particularly useful when we have an initial solution that is dual feasible but primal infeasible.

Key features of our implementation:

1. Efficient basis inverse updates using the Sherman-Morrison-Woodbury formula
2. Detailed iteration history for analysis and visualization
3. Robust handling of numerical issues with tolerance parameters
4. Visualization of the solution path for 2D problems

The revised dual simplex method is an important algorithm in the toolkit of linear optimization techniques, complementing the primal simplex method and interior point methods.